#Load data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dialogues_018.json to dialogues_018.json


In [1]:
# intents
!mkdir -p dialogue_018/NONE
!mkdir -p dialogue_018/SearchOnewayFlight
!mkdir -p dialogue_018/ReserveOnewayFlight
!mkdir -p dialogue_018/SearchRoundtripFlights
!mkdir -p dialogue_018/ReserveRoundtripFlights

Script para coletar apenas as utterances (frases) do usuário e armazená-las num diretório que representa a intenção(que será nossa classe para a tarefa de classificação de intenção).</br>
<b>atenção</b>: O sistema foi excluído da coleta de utterances porque a resposta do sistema não possui intenção

In [3]:
import json

if __name__ == '__main__':
    with open("dialogues_018.json", 'r') as file:
        data_018 = json.load(file)

    for i in range(len(data_018)):
        frames_amount = len(data_018[i]['turns'])
        for j in range(frames_amount):
            current_frame = data_018[i]['turns'][j]
            speaker = current_frame['speaker']
            if speaker == 'USER':
                intent = current_frame['frames'][0]['state']['active_intent']
                utterance = current_frame['utterance']

                file_path = "dialogue_018/" + intent + "/" + str(i) + "_" + str(j) + ".txt"

                f = open(file_path, "w+")
                f.write(utterance)
                f.close()


Armazenar numa estrutura para treinamento no Keras

In [2]:
import tensorflow as tf

seed = 42

raw_train_ds= tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    validation_split=0.2,
    subset='training',
    seed=seed,
    shuffle=False)

raw_validation_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    validation_split=0.2,
    subset='validation',
    seed=seed,
    shuffle=False)

Found 1052 files belonging to 5 classes.
Using 842 files for training.
Found 1052 files belonging to 5 classes.
Using 210 files for validation.


In [25]:
raw_train_ds.class_names

['NONE',
 'ReserveOnewayFlight',
 'ReserveRoundtripFlights',
 'SearchOnewayFlight',
 'SearchRoundtripFlights']

In [ ]:
for text_batch, label_batch in raw_train_ds:
  print("Utterance: ", text_batch)
  print("Label: ", label_batch)

In [27]:
print(raw_train_ds)

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


Acho que o Tensor acima é dividido em batches e acredito que isso adiciona uma complexidade na hora de tokenizar, pois teríamos que fazer para cada um dos tensores. Testei resolver fazer uma concatenação de todos os tensores abaixo:

In [4]:
all_text_tensor = tf.concat([text_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_text_tensor

In [23]:
all_labels = tf.concat([label_batch for text_batch, label_batch in raw_train_ds], axis=0)
#all_labels

In [27]:
utterances_validation_tensor = tf.concat([text_batch for text_batch, label_batch in raw_validation_ds], axis=0)

In [28]:
validation_labels = tf.concat([label_batch for text_batch, label_batch in raw_validation_ds], axis=0)

# Pre processamento de dados
Fazer uma bag-of-words

In [11]:
from keras.preprocessing.text import Tokenizer
import json
import numpy

In [15]:
from keras.preprocessing import text, sequence

### Train subset

In [14]:
t = Tokenizer()
texts = all_text_tensor.numpy().astype(str)
t.fit_on_texts(texts)

# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(texts, mode='count')
print(encoded_docs)

OrderedDict([('no', 73), ('that', 175), ('will', 62), ('be', 70), ('all', 75), ('thank', 75), ('you', 191), ("that's", 68), ('thanks', 92), ('for', 217), ('your', 54), ('help', 101), ('nope', 8), ("we're", 1), ('good', 101), ('a', 93), ('lot', 5), ('the', 280), ('is', 114), ("i'm", 46), ('right', 17), ('though', 1), ('thats', 8), ('it', 54), ('i', 282), ('just', 8), ('needed', 8), ('information', 2), ('now', 11), ('again', 4), ('everything', 13), ('would', 50), ('very', 10), ('much', 16), ('do', 17), ('not', 5), ('want', 54), ('yet', 1), ('was', 1), ('okay', 11), ('this', 63), ('should', 1), ('enough', 1), ('to', 225), ('know', 4), ('maybe', 1), ('another', 6), ('time', 24), ("that'll", 1), ('wanted', 2), ('but', 6), ('am', 62), ('nothing', 1), ('else', 7), ('set', 1), ('yes', 122), ('can', 93), ('assist', 3), ('me', 145), ('with', 14), ('buying', 3), ('ticket', 67), ('correct', 20), ('yeah', 3), ('perfect', 24), ('great', 34), ('sure', 2), ('go', 16), ('ahead', 4), ('and', 125), ('buy

In [16]:
seq_token = t.texts_to_sequences(texts)
padded_sequences = sequence.pad_sequences(seq_token)

In [17]:
padded_sequences.shape

(842, 30)

In [34]:
all_labels.shape

TensorShape([842])

### Validation subset

In [47]:
t2 = Tokenizer()
validation_texts = utterances_validation_tensor.numpy().astype(str)
t2.fit_on_texts(validation_texts)
validation_seq_token = t2.texts_to_sequences(validation_texts)
padded_validation = sequence.pad_sequences(validation_seq_token)
#padded_validation = sequence.pad_sequences(validation_seq_token, maxlen=30)

In [48]:
padded_validation.shape

(210, 28)

In [45]:
validation_labels.shape

TensorShape([210])

# Classificação de intenções (Parte 1)
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM </br>
https://keras.io/examples/nlp/text_classification_with_transformer/ </br>

## LSTM

In [18]:
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM
from keras.layers import Embedding
import keras.backend as K

model = Sequential()
model.add(Embedding(input_dim=842, output_dim=100))
model.add(LSTM(units=64))
model.add(Dense(5,activation='softmax'))
model.compile(loss=['sparse_categorical_crossentropy'] , optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         84200     
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 126765 (495.18 KB)
Trainable params: 126765 (495.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
history = model.fit(padded_sequences, all_labels, epochs=10)

Epoch 1/10
27/27 [==============================] - 3s 23ms/step - loss: 1.3810 - accuracy: 0.4382
Epoch 2/10
27/27 [==============================] - 1s 22ms/step - loss: 1.1377 - accuracy: 0.6152
Epoch 3/10
27/27 [==============================] - 1s 23ms/step - loss: 0.8677 - accuracy: 0.7209
Epoch 4/10
27/27 [==============================] - 1s 36ms/step - loss: 0.7142 - accuracy: 0.7423
Epoch 5/10
27/27 [==============================] - 1s 36ms/step - loss: 0.5947 - accuracy: 0.8029
Epoch 6/10
27/27 [==============================] - 1s 37ms/step - loss: 0.5250 - accuracy: 0.8325
Epoch 7/10
27/27 [==============================] - 1s 21ms/step - loss: 0.4483 - accuracy: 0.8492
Epoch 8/10
27/27 [==============================] - 1s 23ms/step - loss: 0.4084 - accuracy: 0.8539
Epoch 9/10
27/27 [==============================] - 1s 22ms/step - loss: 0.3680 - accuracy: 0.8741
Epoch 10/10
27/27 [==============================] - 1s 23ms/step - loss: 0.3447 - accuracy: 0.8836


In [ ]:
history = model.fit(padded_sequences, all_labels, validation_data=(padded_validation, validation_labels), epochs=10)

## SVM

## CNN

## Transformer
implementar incialmente exatamente como está na documentação e depois testar mudar os parâmetros

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Implement a transformer block as a layer

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Implement embedding layer

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions